In [5]:
%%writefile ../langgraph.json
{
    "dependencies": [
        "langchain",
        "langchain-openai", 
        "langchain-community",
        "langchain-huggingface",
        "langgraph",
        "qdrant-client",
        "python-dotenv",
        "pypdf",
        "rich",
        "."
    ],
    "graphs": {
        "space_biology_agent": "./src/studio_agent.py:graph"
    },
    "env": ".env",
    "dockerfile_lines": [
        "RUN pip install --no-cache-dir -e ."
    ]
}

Overwriting ../langgraph.json


In [6]:
%%writefile ../setup.py
"""
Setup configuration for Space Biology Knowledge Engine
"""

from setuptools import setup, find_packages

setup(
    name="space-biology-knowledge-engine",
    version="0.1.0",
    description="AI-powered space biology research assistant",
    packages=find_packages(),
    install_requires=[
        "langchain>=0.2.16",
        "langchain-openai>=0.1.23",
        "langchain-community>=0.2.16",
        "langchain-huggingface>=0.0.3",
        "langgraph>=0.2.14",
        "qdrant-client>=1.10.1",
        "python-dotenv>=1.0.1",
        "pypdf>=4.3.1",
        "rich>=13.7.1"
    ],
    python_requires=">=3.9",
)

Writing ../setup.py


In [3]:
%%writefile ../src/studio_agent.py

"""
LangGraph Studio Entry Point for Space Biology Knowledge Engine

This file provides the graph object that LangGraph Studio expects.
"""

from langgraph.checkpoint.memory import InMemorySaver
from src.master_research_workflow import create_master_research_workflow

# Create the graph with checkpointer for Studio
def create_studio_graph():
    """Create graph compatible with LangGraph Studio."""
    
    # Create checkpointer for conversation memory
    checkpointer = InMemorySaver()
    
    # Create and compile the workflow with checkpointer
    workflow = create_master_research_workflow()
    
    # Note: If your workflow is already compiled, we need to rebuild with checkpointer
    from langgraph.graph import StateGraph, START, END
    from src.master_state import MasterResearchState
    from src.research_orchestrator import (
        research_orchestrator,
        scoping_node, 
        retrieval_node,
        synthesis_node,
        route_next_step
    )
    
    # Rebuild workflow with checkpointer
    studio_workflow = StateGraph(MasterResearchState)
    
    # Add nodes
    studio_workflow.add_node("orchestrator", research_orchestrator)
    studio_workflow.add_node("scoping", scoping_node)
    studio_workflow.add_node("retrieval", retrieval_node)
    studio_workflow.add_node("synthesis", synthesis_node)
    
    # Add edges
    studio_workflow.add_edge(START, "orchestrator")
    studio_workflow.add_conditional_edges(
        "orchestrator",
        route_next_step,
        {
            "scoping": "scoping",
            "retrieval": "retrieval", 
            "synthesis": "synthesis",
            "__end__": END
        }
    )
    studio_workflow.add_edge("scoping", "orchestrator")
    studio_workflow.add_edge("retrieval", "orchestrator") 
    studio_workflow.add_edge("synthesis", END)
    
    return studio_workflow.compile(checkpointer=checkpointer)

# Create the graph instance that Studio expects
graph = create_studio_graph()

Writing ../src/studio_agent.py


In [4]:
%%writefile ../requirements.txt
langchain==0.2.16
langchain-openai==0.1.23
langchain-community==0.2.16
langchain-huggingface==0.0.3
langgraph==0.2.14
qdrant-client==1.10.1
python-dotenv==1.0.1
pypdf==4.3.1
rich==13.7.1
jupyter==1.0.0

Writing ../requirements.txt
